qgrid - SlickGrid in Jupyter Notebooks
======================================
Qgrid is an Jupyter notebook widget which uses a javascript library called SlickGrid to render pandas DataFrames within a Jupyter notebook. It was developed for use in [Quantopian's hosted research environment]( https://www.quantopian.com/research?utm_source=github&utm_medium=web&utm_campaign=qgrid-nbviewer).

The purpose of this notebook is to give an overview of what qgrid is capable of. Execute the cells below to generate some qgrids using a diverse set of DataFrames.

In case you're wondering why every cell has redundant `import` statements, it's because it's convenient to not have to run an `import` cell at the top of the notebook every time the kernel is restarted. This is mainly useful during development...in practice you'll probably be happy just having a single import cell at the top of your notebook.

## Overview
* [SlickGrid](https://github.com/mleibman/SlickGrid) is a javascript grid which allows users to scroll, sort, 
and filter hundreds of thousands of rows with extreme responsiveness.  
* [Pandas](https://github.com/pydata/pandas) is a powerful data analysis / manipulation library for Python, and DataFrames are the primary way of storing and manipulating two-dimensional data in pandas.

[Qgrid](https://github.com/quantopian/qgrid) renders pandas DataFrames as SlickGrids, which enables users to explore the entire contents of a DataFrame using intuitive sorting and filtering controls.  It's built on the ipywidget designed to be used with Jupyter notebook, Jupyterhub, or Jupyterlab

## What's new in 1.0.0
* Editing by double clicking is always enabled, and a version of your DataFrame that includes your edits can be retrieved by calling `get_changed_df` on a `QgridWidget` instance (which can be obtained by constructing it directly or by calling `show_grid`)
* Editing is supported for all non-index columns other than interval-type columns
* Filtering and sorting changes will also be reflected in the DataFrame returned by `get_changed_df`
* Support for boolean columnns, which are empty when `False` and filled with a checkmark when `True`
* Compatible with ipywidgets 7, and works with Jupyterhub and Jupyterlab
* Limited support for the new interval column type in pandas (editing is not allowed)
* "Full screen" button which is available when the `show_toolbar` option is set to `True`
* Increased scale: moving filtering/sorting/editing logic to the server means qgrid can handle millions of rows easily
* `enable`/`disable` methods to control whether DataFrames and Series are automatically rendered as qgrids for the lifetime of the kernel

## API & Usage
API documentation is hosted on [readthedocs](http://qgrid.readthedocs.io/en/widget-guidelines/). 

The API documentation can also be accessed via the "?" operator in IPython.  To use the "?" operator, type the name of the function followed by "?" to see the documentation for that function, like this:
```
qgrid.show_grid?
qgrid.set_defaults?
qgrid.set_grid_options?
qgrid.enable?
qgrid.disable?

```

## Example 1 - Render a DataFrame with many different types of columns

In [51]:
import numpy as np
import pandas as pd
import qgrid
randn = np.random.randn
df_types = pd.DataFrame({
    'A' : 1.,
    'B' : pd.Series(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08', '2013-01-09'],index=list(range(9)),dtype='datetime64[ns]'),
    'C' : pd.Series(randn(9),index=list(range(9)),dtype='float32'),
    'D' : np.array([3] * 9,dtype='int32'),
    'E' : pd.Categorical(["washington", "adams", "washington", "madison", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'F' : ["foo", "bar", "buzz", "bippity","boppity", "foo", "foo", "bar", "zoo"] })
df_types['G'] = df_types['F'] == 'foo'
qgrid_widget = qgrid.QgridWidget(df=df_types, show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 10, 'minVisibleRows': 8}, precision=5, show_toolbar=True)

If you make any sorting/filtering changes, or edit the grid by double clicking, you can retrieve a copy of your DataFrame which reflects these changes by calling `get_changed_df` on the `QgridWidget` instance returned by `show_grid`.

In [52]:
qgrid_widget.get_changed_df()

,A,B,C,D,E,F,G
0,1.0,2013-01-01,0.649746,3,washington,foo,True
1,1.0,2013-01-02,-0.262323,3,adams,bar,False
2,1.0,2013-01-03,0.032397,3,washington,buzz,False
3,1.0,2013-01-04,1.120246,3,madison,bippity,False
4,1.0,2013-01-05,-0.056222,3,lincoln,boppity,False
5,1.0,2013-01-06,0.036962,3,jefferson,foo,True
6,1.0,2013-01-07,-0.127180,3,hamilton,foo,True
7,1.0,2013-01-08,-1.313404,3,roosevelt,bar,False
8,1.0,2013-01-09,1.789556,3,kennedy,zoo,False


## Example 2 - Render a DataFrame with 1 million rows

In [53]:
import pandas as pd
import numpy as np
import qgrid

# set the default max number of rows to 10 so the larger DataFrame we render don't take up to much space 
qgrid.set_grid_option('maxVisibleRows', 10)

df_scale = pd.DataFrame(np.random.randn(1000000, 4), columns=list('ABCD'))
# duplicate column B as a string column, to test scalability for text column filters
df_scale['B (as str)'] = df_scale['B'].map(lambda x: str(x))
qgrid.show_grid(df_scale, show_toolbar=True, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defaultColumnWidth': 200, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 10, 'minVisibleRows': 8}, precision=5, show_toolbar=True)

## Example 3 - Render a DataFrame returned by Yahoo Finance

In [54]:
import pandas as pd
import numpy as np
import qgrid
randn = np.random.randn

# Get a pandas DataFrame containing the daily prices for the S&P 500 from 1/1/2014 - 1/1/2017
from pandas_datareader.data import DataReader
spy = DataReader(
    'SPY',
    'yahoo',
    pd.Timestamp('2014-01-01'),  
    pd.Timestamp('2017-01-01'),
)
# Construct a QgridWidget instance directly instead of using 'show_grid'
qgrid.QgridWidget(df=spy)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 10, 'minVisibleRows': 8}, precision=5)

## Example 4 - Render a DataFrame with a multi-index

Create a sample DataFrame using the `wb.download` function and render it without using qgrid

In [55]:
import qgrid
import pandas as pd
from pandas_datareader import wb
df_countries = wb.download(indicator='NY.GDP.PCAP.KD', country=['all'], start=2005, end=2008)
df_countries.columns = ['GDP per capita (constant 2005 US$)']
qgrid.show_grid(df_countries)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 10, 'minVisibleRows': 8}, precision=5)

## Example 5 - Render a DataFrame with an interval column
Create a sample DataFrame using the `wb.download` function and render it without using qgrid

In [56]:
import numpy as np
import pandas as pd
import qgrid

td = np.cumsum(np.random.randint(1, 15*60, 1000))
start = pd.Timestamp('2017-04-17')
df_interval = pd.DataFrame(
    [(start + pd.Timedelta(seconds=d)) for d in td],
    columns=['time'])

freq = '15Min'
start = df_interval['time'].min().floor(freq)
end = df_interval['time'].max().ceil(freq)
bins = pd.date_range(start, end, freq=freq)

df_interval['time_bin'] = pd.cut(df_interval['time'], bins)

qgrid.QgridWidget(df=df_interval, show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 10, 'minVisibleRows': 8}, precision=5, show_toolbar=True)

## Example 6 - Render a DataFrame with unnamed columns
Create a sample DataFrame using the `wb.download` function and render it without using qgrid

In [57]:
import numpy as np
import pandas as pd
import qgrid

arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
df_multi = pd.DataFrame(np.random.randn(8, 4), index=arrays)
qgrid.show_grid(df_multi)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 10, 'minVisibleRows': 8}, precision=5)